In [86]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import sys
import time

import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.
def data_type(): return tf.float32
train_size = train_labels.shape[0]

Training: (1) Preparing Data:

In [82]:
data_dir = 'data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

train_data = mnist.train.images
train_labels = mnist.train.labels
validation_data  = mnist.validation.images
validation_labels = mnist.validation.labels

train_data = train_data.reshape(train_data.shape[0], IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
validation_data = validation_data.reshape(validation_data.shape[0], IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


Training: (2) Define Input placeholder and weight variables

In [93]:
#train_data_node = tf.placeholder(
#      data_type(),
#      shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
#train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE, NUM_LABELS))

train_data_node = tf.placeholder(
      data_type(),
      shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(None, NUM_LABELS))

conv1_weights = tf.Variable(
      tf.truncated_normal([5, 5, NUM_CHANNELS, 32],  # 5x5 filter, depth 32.
                          stddev=0.1,
                          seed=SEED, dtype=data_type()))
conv1_biases = tf.Variable(tf.zeros([32], dtype=data_type()))
conv2_weights = tf.Variable(tf.truncated_normal(
      [5, 5, 32, 64], stddev=0.1,
      seed=SEED, dtype=data_type()))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=data_type()))
fc1_weights = tf.Variable(  # fully connected, depth 512.
      tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 512],
                          stddev=0.1,
                          seed=SEED,
                          dtype=data_type()))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=data_type()))
fc2_weights = tf.Variable(tf.truncated_normal([512, NUM_LABELS],
                                                stddev=0.1,
                                                seed=SEED,
                                                dtype=data_type()))
fc2_biases = tf.Variable(tf.constant(
      0.1, shape=[NUM_LABELS], dtype=data_type()))

Training (3): Define the Graph

In [98]:
conv = tf.nn.conv2d(train_data_node,
                    conv1_weights,
                    strides=[1, 1, 1, 1],
                    padding='SAME')
relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
pool = tf.nn.max_pool(relu,
                    ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1],
                    padding='SAME')
conv = tf.nn.conv2d(pool,
                    conv2_weights,
                    strides=[1, 1, 1, 1],
                    padding='SAME')
relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
pool = tf.nn.max_pool(relu,
                    ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1],
                    padding='SAME')
pool_shape = pool.get_shape().as_list()
reshape = tf.reshape(
    pool,
    #[pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]]
    [tf.shape(pool)[0], pool_shape[1] * pool_shape[2] * pool_shape[3]]
)
hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
# if train:
hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
logits = tf.matmul(hidden, fc2_weights) + fc2_biases

Training (4): Define Training parameters

In [99]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      labels=train_labels_node, logits=logits))
#loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
#      labels=train_labels_node, logits=logits))
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
loss += 5e-4 * regularizers
batch = tf.Variable(0, dtype=data_type())
learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch * BATCH_SIZE,  # Current index into the dataset.
      train_labels.shape[0],          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
prediction = tf.nn.softmax(logits)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(train_labels_node, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Traing (5): Start Training & Validation

In [ ]:
start_time = time.time()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        #batch_data, batch_labels = mnist.train.next_batch(BATCH_SIZE)
        feed_dict = {train_data_node: batch_data,
                    train_labels_node: batch_labels}
        # Run the optimizer to update weights.
        sess.run(optimizer, feed_dict=feed_dict)
        # print some extra information once reach the evaluation frequency
        if step % EVAL_FREQUENCY == 0:
            # fetch some extra nodes' data
            l, lr, predictions, acc = sess.run([loss, learning_rate, prediction, accuracy],
                                      feed_dict=feed_dict)
            elapsed_time = time.time() - start_time
            start_time = time.time()
            print('Step %d (epoch %.2f), %.1f ms' %
                  (step, float(step) * BATCH_SIZE / train_size,
                   1000 * elapsed_time / EVAL_FREQUENCY))
            print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            print('Minibatch error: %.1f%%' %  acc)
            print('Validation error: %.1f%%' % accuracy.eval(feed_dict={
                train_data_node: validation_data,
                train_labels_node: validation_labels
            }))
            
            # error_rate(
            #    eval_in_batches(validation_data, sess), validation_labels))
            sys.stdout.flush()

Step 0 (epoch 0.00), 7.1 ms
Minibatch loss: 8.280, learning rate: 0.010000
Minibatch error: 0.2%
Validation error: 0.1%
